# Tools padrão do LangChain

A biblioteca LangChain já possui diversas ferramentas padrão construídas que podemos utilizar. Elas podem ser verificadas neste link: https://python.langchain.com/docs/integrations/tools/.
Vamos mostrar como utilizar algumas.

## ArXiv

Esta ferramenta utiliza a biblioteca do arXiv para retornar resumos de artigos científicos de um determinado tema solicitado.

https://python.langchain.com/v0.1/docs/integrations/tools/arxiv/

Temos três formas em geral para chamar uma tool, da mais baixo a mais alto nível:
- Criando a tool utilziando o Wrapper: no LangChain foram criados diferentes wrappers para apis e bibliotecas externas que facilitam a utilização das mesmas e permitem uma rápida criação de uma nova tool. Ela é mais customizável, pois podemos modificar as descrições e argumentos da tool às nossa necessidades, além de podermos modificar facilmente os parâmetros do wrapper
- Instanciando a tool já criada: em geral, uma tool pronta já está criada dentro da biblioteca e podemos acessá-la diretamente
- Utilizando o load_tools: o LangChain possui uma ferramnta especial chamada load_tools que permite o carregamento de diversas ferramentas ao mesmo tempo

Vamos explorar os três métodos agora:

### Criando tool manualmente através do Wrapper

In [3]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool

class ArxivArgs(BaseModel):
    query: str = Field(description='Query de busca no ArXiv')

tool_arxiv = StructuredTool.from_function(
    func=ArxivAPIWrapper(top_k_results=2).run,
    args_schema=ArxivArgs,
    name='arxiv',
    description=(
        "Uma ferramenta em torno do Arxiv.org. "
        "Útil para quando você precisa responder a perguntas sobre Física, Matemática, "
        "Ciência da Computação, Biologia Quantitativa, Finanças Quantitativas, Estatística, "
        "Engenharia Elétrica e Economia utilizando artigos científicos do arxiv.org. "
        "A entrada deve ser uma consulta de pesquisa em inglês."
    ),
    return_direct=True
)


In [4]:
print('Descrição: ', tool_arxiv.description)
print('Args: ', tool_arxiv.args)

Descrição:  Uma ferramenta em torno do Arxiv.org. Útil para quando você precisa responder a perguntas sobre Física, Matemática, Ciência da Computação, Biologia Quantitativa, Finanças Quantitativas, Estatística, Engenharia Elétrica e Economia utilizando artigos científicos do arxiv.org. A entrada deve ser uma consulta de pesquisa em inglês.
Args:  {'query': {'description': 'Query de busca no ArXiv', 'title': 'Query', 'type': 'string'}}


In [5]:
tool_arxiv.run({'query': 'Machine Learning'})

"Published: 2019-09-08\nTitle: Lecture Notes: Optimization for Machine Learning\nAuthors: Elad Hazan\nSummary: Lecture notes on optimization for machine learning, derived from a course at\nPrinceton University and tutorials given in MLSS, Buenos Aires, as well as\nSimons Foundation, Berkeley.\n\nPublished: 2018-11-11\nTitle: An Optimal Control View of Adversarial Machine Learning\nAuthors: Xiaojin Zhu\nSummary: I describe an optimal control view of adversarial machine learning, where the\ndynamical system is the machine learner, the input are adversarial actions, and\nthe control costs are defined by the adversary's goals to do harm and be hard\nto detect. This view encompasses many types of adversarial machine learning,\nincluding test-item attacks, training-data poisoning, and adversarial reward\nshaping. The view encourages adversarial machine learning researcher to utilize\nadvances in control theory and reinforcement learning."

### Instanciando tool já criada no LangChain

In [6]:
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(top_k_results=2))

In [7]:
print('Descrição: ', tool_arxiv.description)
print('Args: ', tool_arxiv.args)

Descrição:  A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.
Args:  {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Criando tool através do load_tools

In [8]:
from langchain.agents import load_tools

In [10]:
tools = load_tools(['arxiv'])
tool_arxiv = tools[0]
tool_arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000))

In [11]:
print('Descrição: ', tool_arxiv.description)
print('Args: ', tool_arxiv.args)

Descrição:  A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.
Args:  {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


In [12]:
tool_arxiv.run({'query': 'Machine Learning'})

"Published: 2019-09-08\nTitle: Lecture Notes: Optimization for Machine Learning\nAuthors: Elad Hazan\nSummary: Lecture notes on optimization for machine learning, derived from a course at\nPrinceton University and tutorials given in MLSS, Buenos Aires, as well as\nSimons Foundation, Berkeley.\n\nPublished: 2018-11-11\nTitle: An Optimal Control View of Adversarial Machine Learning\nAuthors: Xiaojin Zhu\nSummary: I describe an optimal control view of adversarial machine learning, where the\ndynamical system is the machine learner, the input are adversarial actions, and\nthe control costs are defined by the adversary's goals to do harm and be hard\nto detect. This view encompasses many types of adversarial machine learning,\nincluding test-item attacks, training-data poisoning, and adversarial reward\nshaping. The view encourages adversarial machine learning researcher to utilize\nadvances in control theory and reinforcement learning.\n\nPublished: 2017-07-16\nTitle: Minimax deviation str

## Python REPL

https://python.langchain.com/v0.1/docs/integrations/tools/python/

In [14]:
from langchain_experimental.tools.python.tool import PythonREPLTool

tool_repl = PythonREPLTool()
print('Descrição: ', tool_repl.description)
print('Args: ', tool_repl.args)

Descrição:  A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.
Args:  {'query': {'title': 'Query', 'type': 'string'}}


In [15]:
tool_repl.run({'query': 'print("oi")'})

Python REPL can execute arbitrary code. Use with caution.


'oi\n'

## StackOverflow

https://python.langchain.com/v0.1/docs/integrations/tools/stackexchange/

In [17]:
from langchain.agents import load_tools

tools = load_tools({'stackexchange'})
tools_stack = tools[0]
print('Descrição: ', tools_stack.description)
print('Args: ', tools_stack.args)

Descrição:  A wrapper around StackExchange. Useful for when you need to answer specific programming questionscode excerpts, code examples and solutionsInput should be a fully formed question.
Args:  {'query': {'title': 'Query', 'type': 'string'}}


In [19]:
tools_stack.run({'query': 'langchain Agents'})

'Question: Trying to get my agent_executor for a sql agent in langchain to accept Prompts and Chat History\nThe <span class="highlight">langchain</span>, agent_executor, SQLDatabaseToolkit all working but I want to prompt it and keep chat history for follow up questions. &hellip; memory_key=&quot;chat_history&quot;)\n\nagent_executor = create_sql_agent(\nllm=llm,\ntoolkit=toolkit,\nverbose=True,\nmemory=memory\n)\n\nagent_executor.run(&quot;How many products bought in Chicago&quot;)\n\nCan i do that with this <span class="highlight">agent</span> &hellip; \n\nQuestion: Need Help to integrate open source LLM in Langgraph\nI know I can use <span class="highlight">Langchain</span> but I want to create a flow of AI <span class="highlight">agents</span>. Can anyone help me. You can help me with code of how to integrate open source llm or prove me with open ai api key. &hellip; \n\nQuestion: Problem with query on database using LangGraph\nI have implemented the SQL <span class="highlight">age

## File System

https://python.langchain.com/v0.1/docs/integrations/tools/filesystem/

In [ ]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
    root_dir='arquivos'
)

tools = tool_kit.get_tools()
for tool in tools:
    print('Nome: ', tool.name)
    print('Descrição: ', tool.description)
    print('Args: ', tool.args)
    print()

Nome:  copy_file
Descrição:  Create a copy of a file in a specified location
Args:  {'source_path': {'description': 'Path of the file to copy', 'title': 'Source Path', 'type': 'string'}, 'destination_path': {'description': 'Path to save the copied file', 'title': 'Destination Path', 'type': 'string'}}

Nome:  file_delete
Descrição:  Delete a file
Args:  {'file_path': {'description': 'Path of the file to delete', 'title': 'File Path', 'type': 'string'}}

Nome:  file_search
Descrição:  Recursively search for files in a subdirectory that match the regex pattern
Args:  {'dir_path': {'default': '.', 'description': 'Subdirectory to search in.', 'title': 'Dir Path', 'type': 'string'}, 'pattern': {'description': 'Unix shell regex, where * matches everything.', 'title': 'Pattern', 'type': 'string'}}

Nome:  move_file
Descrição:  Move or rename a file from one location to another
Args:  {'source_path': {'description': 'Path of the file to move', 'title': 'Source Path', 'type': 'string'}, 'destin

## Dando um exemplo de aplicação

In [37]:
from langchain_community.agent_toolkits.file_management.toolkit import FileManagementToolkit

tool_kit = FileManagementToolkit(
    root_dir='arquivos',
    selected_tools=['write_file', 'read_file', 'file_search', 'list_directory']
)
tools = tool_kit.get_tools()

In [38]:
from langchain.prompts import ChatPromptTemplate as cpt
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from dotenv import load_dotenv, find_dotenv

_=load_dotenv(find_dotenv())

tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

prompt = cpt.from_messages([
    ('system', 'Você é um assistente amigável capaz de gerenciar arquivos'),
    ('user', '{input}')
])
chat = ChatOpenAI(model='gpt-4o-mini')

In [46]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [47]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [48]:
chain.invoke({'input': 'O que você é capaz de fazer?'})

'Eu sou um assistente que pode gerenciar arquivos e diretórios. Aqui estão algumas das coisas que posso fazer:\n\n1. **Criar e editar arquivos**: Posso escrever texto em arquivos novos ou existentes.\n2. **Ler arquivos**: Posso ler o conteúdo de arquivos.\n3. **Pesquisar arquivos**: Posso procurar arquivos em diretórios que correspondam a padrões específicos.\n4. **Listar diretórios**: Posso listar arquivos e subdiretórios em um diretório específico.\n\nSe você precisar de ajuda com alguma dessas tarefas, é só me avisar!'

In [ ]:
chain.invoke({'input': 'Quais arquivos pdf você tem na pasta?'})

'Chinook.db\nExplorando o Universo das IAs com Hugging Face.pdf'

In [51]:
chain.invoke({'input': 'Quais arquivos txt você tem na pasta?'})

'No files found for pattern *.txt in directory .'

In [52]:
chain.invoke({'input': 'Crie um arquivo txt chamado notas com o conteúdo informando que esse arquivo foi feito por uma inteligência artificial'})

'File written successfully to notas.txt.'

In [53]:
chain.invoke({'input': 'Leia o arquivo notas.txt'})

'Este arquivo foi criado por uma inteligência artificial.'